In [ ]:
# Below, we show the libraries needed for the economic section.
# Since they are older versions than the ones in the requirements.txt, 
# we do not need to include them in that file. However, for exact reproducity
# we write them here:
# pandas>=2.0.0 
# matplotlib>=3.5.0
# seaborn>=0.12.0
# numpy>=1.24.0
# ipykernel>=6.0.0

In [1]:
# Import required libraries for data manipulation and visualization
import pandas as pd
import matplotlib.pyplot as plt

# Configure pandas display options for better readability
pd.set_option("display.max_columns", 50)  # Show up to 50 columns
pd.set_option("display.width", 140)  # Set display width to 140 characters


# Define path to raw data CSV file
CSV_PATH = r"Receptes_facturades_al_Servei_Català_de_la_Salut_20251121.csv"

In [2]:
# Load the CSV file with correct separators for Spanish/Catalan format
# thousands="." for 1.000.000 format, decimal="," for 1,5 format
df = pd.read_csv(CSV_PATH, thousands=".", decimal=",", low_memory=False)
initial_rows = len(df)
print(f"Data loaded successfully: {initial_rows:,} rows")

Data loaded successfully: 7,828,021 rows


In [3]:
# Examine the data structure: shape, columns, and preview
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Shape: (7828021, 18)

Columns: ['any', 'mes', 'codi de la regió sanitària', 'regió sanitària', "grup d'edat", 'sexe', 'codi del grup ATC nivell 1', 'grup ATC nivell 1', 'codi del grup ATC nivell 2', 'grup ATC nivell 2', 'codi del grup ATC nivell 3', 'grup ATC nivell 3', 'codi del grup ATC nivell 4', 'grup ATC nivell 4', 'nombre de receptes', "nombre d'envasos", 'import íntegre', 'import aportació CatSalut']

First few rows:
    any  mes codi de la regió sanitària regió sanitària grup d'edat  sexe codi del grup ATC nivell 1                 grup ATC nivell 1  \
0  2025    9                         61          LLEIDA     0-1 any  Dona                          A  TRACTO ALIMENTARIO Y METABOLISMO   
1  2025    9                         61          LLEIDA     0-1 any  Dona                          A  TRACTO ALIMENTARIO Y METABOLISMO   
2  2025    9                         61          LLEIDA     0-1 any  Dona                          A  TRACTO ALIMENTARIO Y METABOLISMO   
3  2025    9        

In [4]:
# Check data types to identify any type conversion issues
print("\nData Types:")
print(df.dtypes)
print("\n" + "="*80)


Data Types:
any                             int64
mes                             int64
codi de la regió sanitària     object
regió sanitària                object
grup d'edat                    object
sexe                           object
codi del grup ATC nivell 1     object
grup ATC nivell 1              object
codi del grup ATC nivell 2     object
grup ATC nivell 2              object
codi del grup ATC nivell 3     object
grup ATC nivell 3              object
codi del grup ATC nivell 4     object
grup ATC nivell 4              object
nombre de receptes              int64
nombre d'envasos                int64
import íntegre                float64
import aportació CatSalut     float64
dtype: object



In [5]:
# Check for missing values (NaN) in each column
print("Missing values per column:")
print(df.isnull().sum())
print("\n" + "="*80)
print("\nTotal rows with any missing value:", df.isnull().any(axis=1).sum())

Missing values per column:
any                           0
mes                           0
codi de la regió sanitària    0
regió sanitària               0
grup d'edat                   0
sexe                          0
codi del grup ATC nivell 1    0
grup ATC nivell 1             0
codi del grup ATC nivell 2    0
grup ATC nivell 2             0
codi del grup ATC nivell 3    0
grup ATC nivell 3             0
codi del grup ATC nivell 4    0
grup ATC nivell 4             0
nombre de receptes            0
nombre d'envasos              0
import íntegre                0
import aportació CatSalut     0
dtype: int64

any                           0
mes                           0
codi de la regió sanitària    0
regió sanitària               0
grup d'edat                   0
sexe                          0
codi del grup ATC nivell 1    0
grup ATC nivell 1             0
codi del grup ATC nivell 2    0
grup ATC nivell 2             0
codi del grup ATC nivell 3    0
grup ATC nivell 3             0

# Drop problematic values

In [6]:
# Define problematic categorical values to remove
# These represent unspecified or "other" categories that reduce data quality
bad_values = ["Sense especificar", "Altres", "ALTRES", "SENSE ESPECIFICAR", "VARIOS"]

# Remove rows with missing values (NaN)
df = df.dropna()

# Remove rows containing any of the problematic values in any column
df = df[~df.isin(bad_values).any(axis=1)]

In [7]:
# Display statistics on removed rows
rows_after_bad_values = len(df)
print(f"Rows after removing bad values: {rows_after_bad_values:,}")
print(f"Rows removed: {initial_rows - rows_after_bad_values:,}")

Rows after removing bad values: 7,536,372
Rows removed: 291,649


In [ ]:
# Convert region code to numeric type (it was object/string due to mixed types in raw data)
# errors='coerce' converts invalid values to NaN instead of raising an error
df['codi de la regió sanitària'] = pd.to_numeric(df['codi de la regió sanitària'], errors='coerce')

# Drop any rows where conversion failed (non-numeric region codes)
df = df[df['codi de la regió sanitària'].notna()]

# Convert to integer type now that all values are valid numbers

df['codi de la regió sanitària'] = df['codi de la regió sanitària'].astype('int64')
print(f"Region code data type: {df['codi de la regió sanitària'].dtype}")

print(f"Rows after converting region code to numeric: {len(df):,}")

Region code data type: int64
Rows after converting region code to numeric: 7,536,372


# Age Group 

In [9]:
# Display unique age group values before normalization

print("Unique age groups before normalization:")
print(df["grup d'edat"].unique())

Unique age groups before normalization:
['0-1 any' '10-11 anys' '12-13 anys' '14-15 anys' '16-17 anys'
 '18-19 anys' '20-24 anys' '2-3 anys' '25-29 anys' '30-34 anys'
 '35-39 anys' '40-44 anys' '45-49 anys' '4-5 anys' '50-54 anys'
 '55-59 anys' '60-64 anys' '65-69 anys' '6-7 anys' '70-74 anys'
 '75-79 anys' '80-84 anys' '8-9 anys' 'Més de 84 anys' '20-24' '25-29'
 '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64' '65-69' '70-74'
 '75-79' '80-84' 'Més de 84']
['0-1 any' '10-11 anys' '12-13 anys' '14-15 anys' '16-17 anys'
 '18-19 anys' '20-24 anys' '2-3 anys' '25-29 anys' '30-34 anys'
 '35-39 anys' '40-44 anys' '45-49 anys' '4-5 anys' '50-54 anys'
 '55-59 anys' '60-64 anys' '65-69 anys' '6-7 anys' '70-74 anys'
 '75-79 anys' '80-84 anys' '8-9 anys' 'Més de 84 anys' '20-24' '25-29'
 '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64' '65-69' '70-74'
 '75-79' '80-84' 'Més de 84']


In [10]:
# Create mapping dictionary to standardize age group labels
# Converts Catalan labels ("anys" = years, "any" = year) to simple numeric ranges
# Also handles duplicate formats and standardizes "Més de 84" to "84+"
mapping = {
    '0-1 any': '0-1', '0-1': '0-1',
    '2-3 anys': '2-3', '2-3': '2-3',
    '4-5 anys': '4-5', '4-5': '4-5',
    '6-7 anys': '6-7', '6-7': '6-7',
    '8-9 anys': '8-9', '8-9': '8-9',
    '10-11 anys': '10-11', '10-11': '10-11',
    '12-13 anys': '12-13', '12-13': '12-13',
    '14-15 anys': '14-15', '14-15': '14-15',
    '16-17 anys': '16-17', '16-17': '16-17',
    '18-19 anys': '18-19', '18-19': '18-19',
    '20-24 anys': '20-24', '20-24': '20-24',
    '25-29 anys': '25-29', '25-29': '25-29',
    '30-34 anys': '30-34', '30-34': '30-34',
    '35-39 anys': '35-39', '35-39': '35-39',
    '40-44 anys': '40-44', '40-44': '40-44',
    '45-49 anys': '45-49', '45-49': '45-49',
    '50-54 anys': '50-54', '50-54': '50-54',
    '55-59 anys': '55-59', '55-59': '55-59',
    '60-64 anys': '60-64', '60-64': '60-64',
    '65-69 anys': '65-69', '65-69': '65-69',
    '70-74 anys': '70-74', '70-74': '70-74',
    '75-79 anys': '75-79', '75-79': '75-79',
    '80-84 anys': '80-84', '80-84': '80-84',
    'Més de 84 anys': '84+', 'Més de 84': '84+', '84+': '84+'
}

# Apply the mapping to normalize all age group labels
# For any values not in the mapping, attempt a simple cleanup:
# remove "anys"/"any" and strip whitespace; if result looks like a known range keep it,
# otherwise preserve original value (so we don't introduce NaNs unexpectedly).
df['grup d\'edat'] = df['grup d\'edat'].map(mapping)

# For unmapped values, try to normalize by removing Catalan year words
unmapped_mask = df['grup d\'edat'].isna()
if unmapped_mask.any():
    cleaned = (df.loc[unmapped_mask, 'grup d\'edat']
               .astype(str)
               .str.replace(r'\banys?\b', '', regex=True)
               .str.strip())
    # If cleaned is non-empty, keep it; otherwise leave original string
    cleaned = cleaned.replace('', pd.NA)
    df.loc[unmapped_mask, 'grup d\'edat'] = cleaned

# Optionally, if you want to ensure no missing age group remain, fill remaining NaNs with a placeholder:
# df['grup d\'edat'] = df['grup d\'edat'].fillna('Unknown')

In [11]:
# Verify age group normalization was successful

print("Unique age groups after normalization:")
print(f"\nTotal unique age groups: {df['grup d\'edat'].nunique()}")
print(sorted(df["grup d'edat"].unique()))

Unique age groups after normalization:

Total unique age groups: 24

Total unique age groups: 24
['0-1', '10-11', '12-13', '14-15', '16-17', '18-19', '2-3', '20-24', '25-29', '30-34', '35-39', '4-5', '40-44', '45-49', '50-54', '55-59', '6-7', '60-64', '65-69', '70-74', '75-79', '8-9', '80-84', '84+']
['0-1', '10-11', '12-13', '14-15', '16-17', '18-19', '2-3', '20-24', '25-29', '30-34', '35-39', '4-5', '40-44', '45-49', '50-54', '55-59', '6-7', '60-64', '65-69', '70-74', '75-79', '8-9', '80-84', '84+']


# Additional Data Cleaning for Economic Analysis

In [12]:
# Check for zero or negative values in economic columns
# 'import íntegre' = total cost, 'import aportació CatSalut' = government payment
print("Checking economic columns for problematic values...")
print(f"\nRows with zero 'import íntegre': {(df['import íntegre'] == 0).sum():,}")
print(f"Rows with negative 'import íntegre': {(df['import íntegre'] < 0).sum():,}")
print(f"Rows with zero 'import aportació CatSalut': {(df['import aportació CatSalut'] == 0).sum():,}")
print(f"Rows with negative 'import aportació CatSalut': {(df['import aportació CatSalut'] < 0).sum():,}")

# Check for data quality issue: government payment exceeding total cost (logically impossible)
print(f"\nRows where CatSalut payment > total cost: {(df['import aportació CatSalut'] > df['import íntegre']).sum():,}")

Checking economic columns for problematic values...

Rows with zero 'import íntegre': 441
Rows with negative 'import íntegre': 0
Rows with zero 'import aportació CatSalut': 176
Rows with negative 'import aportació CatSalut': 0

Rows where CatSalut payment > total cost: 278


In [13]:
# Remove rows with invalid economic values
rows_before_economic_cleaning = len(df)
print(f"\nRows before cleaning economic values: {rows_before_economic_cleaning:,}")

# Remove rows where government payment exceeds total cost (data quality issue)
# This is logically impossible and indicates data entry errors
df = df[df['import aportació CatSalut'] <= df['import íntegre']]
rows_after_economic_cleaning = len(df)
print(f"Rows after cleaning economic values: {rows_after_economic_cleaning:,}")
print(f"Rows removed: {rows_before_economic_cleaning - rows_after_economic_cleaning:,}")

# Calculate the fraction paid by the government (government payment / total cost)
# Handle division by zero by setting fraction to 0 when total cost is 0
df['fracció_govern'] = df.apply(
    lambda row: row['import aportació CatSalut'] / row['import íntegre'] if row['import íntegre'] != 0 else 0, 
    axis=1
)

print("\nFraction paid by government - statistics:")

print(df['fracció_govern'].describe())

print(f"\nRows with zero total cost: {(df['import íntegre'] == 0).sum():,}")


Rows before cleaning economic values: 7,536,372


Rows after cleaning economic values: 7,536,094
Rows removed: 278

Fraction paid by government - statistics:

Fraction paid by government - statistics:
count    7.536094e+06
mean     8.246270e-01
std      1.507254e-01
min      0.000000e+00
25%      6.735902e-01
50%      9.004525e-01
75%      9.404294e-01
max      1.000000e+00
Name: fracció_govern, dtype: float64

Rows with zero total cost: 176
count    7.536094e+06
mean     8.246270e-01
std      1.507254e-01
min      0.000000e+00
25%      6.735902e-01
50%      9.004525e-01
75%      9.404294e-01
max      1.000000e+00
Name: fracció_govern, dtype: float64

Rows with zero total cost: 176


In [14]:
# Inspect value distributions for all categorical columns
# This helps verify data quality and understand the dataset composition
cat_cols = ["any", "mes", "regió sanitària", "grup d'edat", "sexe", "grup ATC nivell 1", "grup ATC nivell 2", "grup ATC nivell 3", "grup ATC nivell 4"]

for col in cat_cols:

    print(f"\n=== {col} ===")    
    print(df[col].value_counts())


=== any ===
any
2024    1076430
2025     811403
2023     731941
2022     722620
2021     708041
2019     703674
2018     700724
2017     697541
2016     693104
2020     690616
Name: count, dtype: int64

=== mes ===
mes
5     655320
3     654939
7     653360
6     651492
4     648528
2     647914
9     647662
1     647365
8     643754
10    563612
11    563369
12    558779
Name: count, dtype: int64

=== regió sanitària ===
any
2024    1076430
2025     811403
2023     731941
2022     722620
2021     708041
2019     703674
2018     700724
2017     697541
2016     693104
2020     690616
Name: count, dtype: int64

=== mes ===
mes
5     655320
3     654939
7     653360
6     651492
4     648528
2     647914
9     647662
1     647365
8     643754
10    563612
11    563369
12    558779
Name: count, dtype: int64

=== regió sanitària ===
regió sanitària
BARCELONA                       1108507
GIRONA                          1104366
CAMP DE TARRAGONA               1051636
CATALUNYA CENTRAL      

In [15]:
# Save the cleaned dataset to a new CSV file
# index=False prevents writing row indices as a column
output_path = r"clean_data.csv"
df.to_csv(output_path, index=False)

print(f"✓ Clean data saved to: {output_path}")

print(f"Final dataset shape: {df.shape}")
print(f"\nColumns in clean dataset: {df.columns.tolist()}")

print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

✓ Clean data saved to: clean_data.csv
Final dataset shape: (7536094, 19)

Columns in clean dataset: ['any', 'mes', 'codi de la regió sanitària', 'regió sanitària', "grup d'edat", 'sexe', 'codi del grup ATC nivell 1', 'grup ATC nivell 1', 'codi del grup ATC nivell 2', 'grup ATC nivell 2', 'codi del grup ATC nivell 3', 'grup ATC nivell 3', 'codi del grup ATC nivell 4', 'grup ATC nivell 4', 'nombre de receptes', "nombre d'envasos", 'import íntegre', 'import aportació CatSalut', 'fracció_govern']
Total rows: 7,536,094
Total columns: 19


## Data Cleaning Summary

This notebook performs comprehensive cleaning of pharmaceutical prescription data from the Servei Català de la Salut (CatSalut).

### Cleaning Steps:

1. **Data Loading**
   - Loaded CSV with proper Spanish/Catalan format separators
   - Decimal separator: comma (,) for values like 1,5
   - Thousands separator: point (.) for values like 1.000.000

2. **Missing Values Removal**
   - Removed all rows with NaN (missing) values
   - Ensures complete records for analysis

3. **Categorical Data Cleaning**

   - Removed problematic categorical values: "Sense especificar", "Altres", "ALTRES", "SENSE ESPECIFICAR", "VARIOS"- **Output file**: `clean_data.csv`

   - These represent unspecified or "other" categories that reduce data quality- **Data quality**: All records complete, validated, and standardized

- **Total columns**: 19 (18 original + 1 engineered feature)

4. **Region Code Standardization**- **Total rows**: 7,535,918 (96.3% of original data retained)

   - Converted region codes from mixed object type to numeric int64### Final Dataset:

   - Removed rows with invalid (non-numeric) region codes

| **Final clean dataset** | **Ready for analysis** | **292,103** | **7,535,918** |

5. **Age Group Normalization**| Economic validation | Invalid economic records (govt payment > total cost) | 454 | 7,535,918 |

   - Standardized all age group labels from Catalan format ("anys", "any") to numeric ranges| Region code conversion | Invalid region codes | - | 7,536,372 |

   - Converted "Més de 84 anys"/"Més de 84" to "84+"| Missing & bad values | NaN and problematic categorical values | 291,649 | 7,536,372 |

   - Resulted in 24 standardized age groups: 0-1, 2-3, 4-5, ..., 80-84, 84+| Initial dataset | Raw data loaded | - | 7,828,021 |

|------|-------------|--------------|----------------|

6. **Economic Data Validation**| Step | Description | Rows Removed | Remaining Rows |

   - Checked for zero or negative values in cost columns

   - Removed invalid records where government payment exceeded total cost (data quality issue)### Cleaning Results:



7. **Feature Engineering**   - Preserved all 19 columns including the new fraction column

   - Created `fracció_govern` column: ratio of government payment to total cost   - Saved cleaned dataset to `clean_data.csv`

   - Handles division by zero by setting fraction to 08. **Data Export**

   - Provides metric for government subsidy analysis